## 问题定义

本问题为求解2D圆柱绕流。假定动力粘性系数$\nu=0.02 kg/(m \cdot s)$，密度$\rho=1.0 kg/m^3$。流体矩形区域长40（-10m - 30m），宽20（-10m - 10m），有一个半径为0.5m的圆柱在$(0 m, 0m)$位置。入口x方向速度为$1.0 m/s$，0压力出口。


### 求解目标

要求求解上述求解区域的稳态解

## 无量纲化

在Modulus中可以使用`NonDimensionalizer`进行无量纲化。


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

import numpy as np
from sympy import Symbol, Eq

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry import Bounds
from modulus.sym.geometry.primitives_2d import Line, Circle, Channel2D
from modulus.sym.eq.pdes.navier_stokes import NavierStokes
from modulus.sym.eq.pdes.basic import NormalDotVec
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)

from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.key import Key
from modulus.sym import quantity
from modulus.sym.eq.non_dim import NonDimensionalizer, Scaler

from modulus.sym.utils.io import (
    ValidatorPlotter,
)

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 200000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 10000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arc

### 定义必要组件

#### 无量纲化

In [4]:
# physical quantities
nu = quantity(0.02, "kg/(m*s)")  # 动力粘性系数
rho = quantity(1.0, "kg/m^3")  # 密度
inlet_u = quantity(1.0, "m/s")  # 入口x方向速度
inlet_v = quantity(0.0, "m/s")  # 入口y方向速度
noslip_u = quantity(0.0, "m/s")  # 无滑移边界x方向速度
noslip_v = quantity(0.0, "m/s")  # 无滑移边界y方向速度
outlet_p = quantity(0.0, "pa")  # 出口压力
velocity_scale = inlet_u  # 以入口x方向速度为基准
density_scale = rho  # 密度基准
length_scale = quantity(20, "m")  # 长度基准
nd = NonDimensionalizer(
    length_scale=length_scale,
    time_scale=length_scale / velocity_scale,
    mass_scale=density_scale * (length_scale**3),
)  # 无量纲化


length scale is 20 meter
time scale is 20.0 second
mass scale is 8000.0 kilogram


#### Geo

In [5]:
channel_length = (quantity(-10, "m"), quantity(30, "m"))  # channel长度
channel_width = (quantity(-10, "m"), quantity(10, "m"))  # channel宽度
cylinder_center = (quantity(0, "m"), quantity(0, "m"))  # 圆柱中心坐标
cylinder_radius = quantity(0.5, "m")  # 圆柱半径
channel_length_nd = tuple(map(lambda x: nd.ndim(x), channel_length))  # 无量纲长度
channel_width_nd = tuple(map(lambda x: nd.ndim(x), channel_width))  # 无量纲宽度
cylinder_center_nd = tuple(map(lambda x: nd.ndim(x), cylinder_center))  # 无量纲中心
cylinder_radius_nd = nd.ndim(cylinder_radius)  # 无量纲圆柱半径

channel = Channel2D(
    (channel_length_nd[0], channel_width_nd[0]),
    (channel_length_nd[1], channel_width_nd[1]),
)  # channel
inlet = Line(
    (channel_length_nd[0], channel_width_nd[0]),
    (channel_length_nd[0], channel_width_nd[1]),
    normal=1,
)  # 入口
outlet = Line(
    (channel_length_nd[1], channel_width_nd[0]),
    (channel_length_nd[1], channel_width_nd[1]),
    normal=1,
)  # 出口
wall_top = Line(
    (channel_length_nd[1], channel_width_nd[0]),
    (channel_length_nd[1], channel_width_nd[1]),
    normal=1,
)
cylinder = Circle(cylinder_center_nd, cylinder_radius_nd)  # 圆柱
volume_geo = channel - cylinder  # 流体区域

#### PDE

In [6]:
ns = NavierStokes(nu=nd.ndim(nu), rho=nd.ndim(rho), dim=2, time=False) 

#### Model

In [7]:
# 定义简单的全连接网络
# 输入为空间坐标和时间
# 输出为方向速度以及压强
flow_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    cfg=cfg.arch.fully_connected,
)

# 法线方向乘以速度
# normal_dot_vel = NormalDotVec(["u", "v"])

x, y = Symbol("x"), Symbol("y")

#### Node

In [8]:
nodes = (
    ns.make_nodes()  # PDE node
#     + normal_dot_vel.make_nodes()  # 法向速度node
    + [flow_net.make_node(name="flow_network")]  # 网络node
    + Scaler(
        ["u", "v", "p"],
        ["u_scaled", "v_scaled", "p_scaled"],
        ["m/s", "m/s", "m^2/s^2"],
        nd,
    ).make_node()  # 无量纲化node
)

#### Domain

In [9]:
domain = Domain()

#### 入口条件

In [10]:
inlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=inlet,
    outvar={"u": nd.ndim(inlet_u), "v": nd.ndim(inlet_v)},
    batch_size=cfg.batch_size.inlet,
)
domain.add_constraint(inlet, "inlet")

#### 出口条件

In [11]:
outlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=outlet,
    outvar={"p": nd.ndim(outlet_p)},
    batch_size=cfg.batch_size.outlet,
)
domain.add_constraint(outlet, "outlet")

#### 滑移边界（channel边界）

In [12]:
walls = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=channel,
    outvar={"u": nd.ndim(inlet_u), "v": nd.ndim(inlet_v)},
    batch_size=cfg.batch_size.walls,
)
domain.add_constraint(walls, "walls")

#### 无滑移边界 （圆柱边界）

In [13]:
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=cylinder,
    outvar={"u": nd.ndim(noslip_u), "v": nd.ndim(noslip_v)},
    batch_size=cfg.batch_size.no_slip,
)
domain.add_constraint(no_slip, "no_slip")

#### PDE约束

In [14]:
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=volume_geo,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.interior,
    bounds=Bounds({x: channel_length_nd, y: channel_width_nd}),
)
domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [15]:
file_path = "openfoam/cylinder_nu_0.020.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {
        "Points:0": "x",
        "Points:1": "y",
        "U:0": "u_scaled",
        "U:1": "v_scaled",
        "p": "p_scaled",
    }
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_invar_numpy = {
        key: value / length_scale.magnitude
        for key, value in openfoam_var.items()
        if key in ["x", "y"]
    }  # 坐标无量纲化
    openfoam_outvar_numpy = {
        key: value
        for key, value in openfoam_var.items()
        if key in ["u_scaled", "v_scaled", "p_scaled"]
    }  # 速度/压强无量纲化
    openfoam_validator = PointwiseValidator(
        nodes=nodes, invar=openfoam_invar_numpy, true_outvar=openfoam_outvar_numpy, plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [16]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [17]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [18]:
slv.solve()

2024-02-28 05:41:30,373 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-02-28 05:41:30,381 - INFO - attempting to restore from: /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 05:41:30,382 - WARNING - optimizer checkpoint not found
2024-02-28 05:41:30,383 - WARNING - model flow_network.0.pth not found
2024-02-28 05:41:32,365 - INFO - [step:          0] record constraint batch time:  1.253e-01s
2024-02-28 05:41:52,568 - INFO - [step:          0] record validators time:  2.020e+01s
2024-02-28 05:41:52,607 - INFO - [step:          0] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 05:41:52,611 - INFO - [step:          0] loss:  5.073e+00
2024-02-28 05:41:55,127 - INFO - Attempting cuda graph building, this may take a bit...
2024-02-28 05:42:15,374 - INFO - [step:        100] loss:  9.731e-02, time/iteration:  2.275e+02 ms
2024-02-28 05:42:27,114 - INFO - [step:        2

2024-02-28 05:56:26,068 - INFO - [step:       6300] loss:  2.611e-02, time/iteration:  1.337e+02 ms
2024-02-28 05:56:41,045 - INFO - [step:       6400] loss:  2.552e-02, time/iteration:  1.498e+02 ms
2024-02-28 05:56:55,125 - INFO - [step:       6500] loss:  2.977e-02, time/iteration:  1.408e+02 ms
2024-02-28 05:57:09,457 - INFO - [step:       6600] loss:  2.962e-02, time/iteration:  1.433e+02 ms
2024-02-28 05:57:21,964 - INFO - [step:       6700] loss:  2.918e-02, time/iteration:  1.251e+02 ms
2024-02-28 05:57:31,165 - INFO - [step:       6800] loss:  2.813e-02, time/iteration:  9.200e+01 ms
2024-02-28 05:57:40,914 - INFO - [step:       6900] loss:  2.709e-02, time/iteration:  9.747e+01 ms
2024-02-28 05:58:13,526 - INFO - [step:       7000] record validators time:  2.061e+01s
2024-02-28 05:58:13,557 - INFO - [step:       7000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 05:58:13,558 - INFO - [step:       7000] loss:  2.792e-02, time/iteration:  3.264e+02 ms
202

2024-02-28 06:11:59,394 - INFO - [step:      13100] loss:  2.824e-02, time/iteration:  1.124e+02 ms
2024-02-28 06:12:09,980 - INFO - [step:      13200] loss:  1.936e-02, time/iteration:  1.058e+02 ms
2024-02-28 06:12:20,814 - INFO - [step:      13300] loss:  1.813e-02, time/iteration:  1.083e+02 ms
2024-02-28 06:12:31,965 - INFO - [step:      13400] loss:  1.872e-02, time/iteration:  1.115e+02 ms
2024-02-28 06:12:47,201 - INFO - [step:      13500] loss:  1.894e-02, time/iteration:  1.523e+02 ms
2024-02-28 06:13:01,013 - INFO - [step:      13600] loss:  1.798e-02, time/iteration:  1.381e+02 ms
2024-02-28 06:13:12,943 - INFO - [step:      13700] loss:  1.914e-02, time/iteration:  1.193e+02 ms
2024-02-28 06:13:23,860 - INFO - [step:      13800] loss:  1.977e-02, time/iteration:  1.092e+02 ms
2024-02-28 06:13:33,993 - INFO - [step:      13900] loss:  1.639e-02, time/iteration:  1.013e+02 ms
2024-02-28 06:14:07,517 - INFO - [step:      14000] record validators time:  2.082e+01s
2024-02-28 0

2024-02-28 06:27:24,465 - INFO - [step:      20000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 06:27:24,467 - INFO - [step:      20000] loss:  8.861e-03, time/iteration:  3.272e+02 ms
2024-02-28 06:27:35,026 - INFO - [step:      20100] loss:  1.316e-02, time/iteration:  1.056e+02 ms
2024-02-28 06:27:44,899 - INFO - [step:      20200] loss:  1.260e-02, time/iteration:  9.872e+01 ms
2024-02-28 06:27:54,692 - INFO - [step:      20300] loss:  1.280e-02, time/iteration:  9.791e+01 ms
2024-02-28 06:28:02,817 - INFO - [step:      20400] loss:  1.321e-02, time/iteration:  8.123e+01 ms
2024-02-28 06:28:10,850 - INFO - [step:      20500] loss:  1.312e-02, time/iteration:  8.032e+01 ms
2024-02-28 06:28:19,122 - INFO - [step:      20600] loss:  1.412e-02, time/iteration:  8.270e+01 ms
2024-02-28 06:28:26,794 - INFO - [step:      20700] loss:  7.803e-03, time/iteration:  7.670e+01 ms
2024-02-28 06:28:35,648 - INFO - [step:      20800] loss:  1.802e-02, time/iteration:  8.85

2024-02-28 06:41:28,050 - INFO - [step:      27000] record validators time:  2.169e+01s
2024-02-28 06:41:28,119 - INFO - [step:      27000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 06:41:28,125 - INFO - [step:      27000] loss:  3.921e-03, time/iteration:  3.517e+02 ms
2024-02-28 06:41:39,695 - INFO - [step:      27100] loss:  6.364e-03, time/iteration:  1.157e+02 ms
2024-02-28 06:41:51,158 - INFO - [step:      27200] loss:  3.762e-03, time/iteration:  1.146e+02 ms
2024-02-28 06:42:02,027 - INFO - [step:      27300] loss:  4.700e-03, time/iteration:  1.087e+02 ms
2024-02-28 06:42:12,955 - INFO - [step:      27400] loss:  3.193e-03, time/iteration:  1.093e+02 ms
2024-02-28 06:42:21,505 - INFO - [step:      27500] loss:  3.589e-03, time/iteration:  8.547e+01 ms
2024-02-28 06:42:29,403 - INFO - [step:      27600] loss:  2.638e-03, time/iteration:  7.895e+01 ms
2024-02-28 06:42:36,906 - INFO - [step:      27700] loss:  8.728e-03, time/iteration:  7.501e+01 ms
202

...

2024-02-28 11:22:26,455 - INFO - [step:     171900] loss:  1.881e-04, time/iteration:  8.809e+01 ms
2024-02-28 11:22:58,224 - INFO - [step:     172000] record validators time:  2.165e+01s
2024-02-28 11:22:58,272 - INFO - [step:     172000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 11:22:58,274 - INFO - [step:     172000] loss:  1.423e-04, time/iteration:  3.182e+02 ms
2024-02-28 11:23:08,821 - INFO - [step:     172100] loss:  1.378e-04, time/iteration:  1.055e+02 ms
2024-02-28 11:23:18,427 - INFO - [step:     172200] loss:  1.561e-04, time/iteration:  9.605e+01 ms
2024-02-28 11:23:26,432 - INFO - [step:     172300] loss:  1.436e-04, time/iteration:  8.004e+01 ms
2024-02-28 11:23:34,677 - INFO - [step:     172400] loss:  1.162e-04, time/iteration:  8.243e+01 ms
2024-02-28 11:23:42,883 - INFO - [step:     172500] loss:  1.534e-04, time/iteration:  8.204e+01 ms
2024-02-28 11:23:50,482 - INFO - [step:     172600] loss:  1.347e-04, time/iteration:  7.598e+01 ms
202

2024-02-28 11:36:03,020 - INFO - [step:     178800] loss:  1.905e-04, time/iteration:  9.351e+01 ms
2024-02-28 11:36:12,314 - INFO - [step:     178900] loss:  1.319e-04, time/iteration:  9.292e+01 ms
2024-02-28 11:36:40,145 - INFO - [step:     179000] record validators time:  1.924e+01s
2024-02-28 11:36:40,187 - INFO - [step:     179000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 11:36:40,190 - INFO - [step:     179000] loss:  1.583e-04, time/iteration:  2.787e+02 ms
2024-02-28 11:36:48,335 - INFO - [step:     179100] loss:  3.051e-04, time/iteration:  8.144e+01 ms
2024-02-28 11:36:55,984 - INFO - [step:     179200] loss:  1.838e-04, time/iteration:  7.648e+01 ms
2024-02-28 11:37:08,960 - INFO - [step:     179300] loss:  1.606e-04, time/iteration:  1.297e+02 ms
2024-02-28 11:37:21,668 - INFO - [step:     179400] loss:  1.427e-04, time/iteration:  1.271e+02 ms
2024-02-28 11:37:32,163 - INFO - [step:     179500] loss:  2.442e-04, time/iteration:  1.049e+02 ms
202

2024-02-28 11:51:35,059 - INFO - [step:     185600] loss:  1.205e-04, time/iteration:  1.104e+02 ms
2024-02-28 11:51:46,339 - INFO - [step:     185700] loss:  1.446e-04, time/iteration:  1.128e+02 ms
2024-02-28 11:51:58,905 - INFO - [step:     185800] loss:  1.375e-04, time/iteration:  1.256e+02 ms
2024-02-28 11:52:11,338 - INFO - [step:     185900] loss:  2.749e-04, time/iteration:  1.243e+02 ms
2024-02-28 11:52:43,739 - INFO - [step:     186000] record validators time:  1.920e+01s
2024-02-28 11:52:43,792 - INFO - [step:     186000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 11:52:43,794 - INFO - [step:     186000] loss:  2.311e-04, time/iteration:  3.245e+02 ms
2024-02-28 11:52:51,684 - INFO - [step:     186100] loss:  1.091e-04, time/iteration:  7.889e+01 ms
2024-02-28 11:52:59,617 - INFO - [step:     186200] loss:  2.072e-04, time/iteration:  7.932e+01 ms
2024-02-28 11:53:08,766 - INFO - [step:     186300] loss:  1.375e-04, time/iteration:  9.147e+01 ms
202

2024-02-28 12:06:02,146 - INFO - [step:     192400] loss:  1.864e-04, time/iteration:  1.140e+02 ms
2024-02-28 12:06:16,043 - INFO - [step:     192500] loss:  1.169e-04, time/iteration:  1.390e+02 ms
2024-02-28 12:06:27,187 - INFO - [step:     192600] loss:  1.411e-04, time/iteration:  1.114e+02 ms
2024-02-28 12:06:40,348 - INFO - [step:     192700] loss:  1.364e-04, time/iteration:  1.316e+02 ms
2024-02-28 12:06:52,042 - INFO - [step:     192800] loss:  9.673e-05, time/iteration:  1.169e+02 ms
2024-02-28 12:07:04,729 - INFO - [step:     192900] loss:  1.439e-04, time/iteration:  1.269e+02 ms
2024-02-28 12:07:37,559 - INFO - [step:     193000] record validators time:  1.990e+01s
2024-02-28 12:07:37,595 - INFO - [step:     193000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-28 12:07:37,597 - INFO - [step:     193000] loss:  1.263e-04, time/iteration:  3.287e+02 ms
2024-02-28 12:07:46,592 - INFO - [step:     193100] loss:  1.530e-04, time/iteration:  8.995e+01 ms
202

2024-02-28 12:18:58,317 - INFO - [step:     199300] loss:  1.213e-04, time/iteration:  9.458e+01 ms
2024-02-28 12:19:06,765 - INFO - [step:     199400] loss:  8.968e-05, time/iteration:  8.446e+01 ms
2024-02-28 12:19:14,644 - INFO - [step:     199500] loss:  1.094e-04, time/iteration:  7.878e+01 ms
2024-02-28 12:19:22,296 - INFO - [step:     199600] loss:  1.076e-04, time/iteration:  7.651e+01 ms
2024-02-28 12:19:30,471 - INFO - [step:     199700] loss:  1.356e-04, time/iteration:  8.174e+01 ms
2024-02-28 12:19:38,225 - INFO - [step:     199800] loss:  1.420e-04, time/iteration:  7.753e+01 ms
2024-02-28 12:19:46,016 - INFO - [step:     199900] loss:  1.125e-04, time/iteration:  7.789e+01 ms
2024-02-28 12:19:56,914 - INFO - [step:     200000] record constraint batch time:  1.665e-01s
2024-02-28 12:20:17,877 - INFO - [step:     200000] record validators time:  2.096e+01s
2024-02-28 12:20:17,923 - INFO - [step:     200000] saved checkpoint to /workspace/14_2D_Cylinder_NS/outputs
2024-02-2

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u_scaled.png)

![v](./outputs/validators/validator_v_scaled.png)

![p](./outputs/validators/validator_p_scaled.png)